In [1]:
# ignore tensorflow messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']= '2'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train),(x_test, y_test)= mnist.load_data()

In [4]:
# -1 is to make interactive, 28, 28 is dimension, 1 is no. of channel
x_train= x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test= x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

In [5]:
# CNN --> BatchNormalization --> relu (common structure)

In [6]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()
        
    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

In [7]:
model = keras.Sequential([CNNBlock(32), CNNBlock(64), CNNBlock(128), layers.Flatten(), layers.Dense(10),])

In [8]:
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x

In [9]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [10]:
model = ResNet_Like().model()
base_input = model.layers[0].input
base_output = model.layers[2].output
output = layers.Dense(10)(layers.Flatten()(base_output))
model = keras.Model(base_input, output)

# or
# model= Rest_Like(num_classes=10)

In [11]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],)

In [12]:
model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)

938/938 - 1638s - loss: 0.1038 - accuracy: 0.9680


In [13]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 60s - loss: 0.0393 - accuracy: 0.9875


[0.03927508369088173, 0.987500011920929]

In [14]:
model.save("keras_subclass")

INFO:tensorflow:Assets written to: keras_subclass\assets


INFO:tensorflow:Assets written to: keras_subclass\assets


In [15]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
res_block (ResBlock)         (None, 14, 14, 64)        28896     
_________________________________________________________________
res_block_1 (ResBlock)       (None, 7, 7, 256)         592512    
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                125450    
Total params: 746,858
Trainable params: 745,578
Non-trainable params: 1,280
_________________________________________________________________


In [12]:
# save model
model.save('keras_subclass/')

INFO:tensorflow:Assets written to: keras_subclass/assets


INFO:tensorflow:Assets written to: keras_subclass/assets
